In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download cdart99/food20dataset

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/cdart99/food20dataset
License(s): CC0-1.0
 98% 428M/435M [00:01<00:00, 261MB/s]
100% 435M/435M [00:01<00:00, 233MB/s]


In [ ]:
! unzip food20dataset.zip

Archive:  food20dataset.zip
  inflating: food20dataset/test_set/biriyani/biriyanitest (1).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (10).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (11).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (12).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (13).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (14).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (15).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (16).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (17).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (18).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (19).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (2).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (20).jpg  
  inflating: food20dataset/test_set/biriyani/biriyanitest (21).jpg  
  inflat

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
        '/content/food20dataset/train_set',
        target_size=(224, 224),
        batch_size=64)
print(train_generator.class_indices)
test_datagen=ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory(
        '/content/food20dataset/test_set',
        target_size=(224, 224),
        batch_size=64)
print(test_generator.class_indices)
print(test_generator.classes)

Found 1400 images belonging to 20 classes.
{'biriyani': 0, 'bisibelebath': 1, 'butternaan': 2, 'chaat': 3, 'chappati': 4, 'dhokla': 5, 'dosa': 6, 'gulab jamun': 7, 'halwa': 8, 'idly': 9, 'kathi roll': 10, 'meduvadai': 11, 'noodles': 12, 'paniyaram': 13, 'poori': 14, 'samosa': 15, 'tandoori chicken': 16, 'upma': 17, 'vada pav': 18, 'ven pongal': 19}
Found 600 images belonging to 20 classes.
{'biriyani': 0, 'bisibelebath': 1, 'butternaan': 2, 'chaat': 3, 'chappati': 4, 'dhokla': 5, 'dosa': 6, 'gulab jamun': 7, 'halwa': 8, 'idly': 9, 'kathi roll': 10, 'meduvadai': 11, 'noodles': 12, 'paniyaram': 13, 'poori': 14, 'samosa': 15, 'tandoori chicken': 16, 'upma': 17, 'vada pav': 18, 'ven pongal': 19}
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  3  3  3  3  3  3
  3  3 

In [ ]:
from tensorflow.keras.applications import InceptionV3
base_model = InceptionV3(
    include_top=False,
    weights = 'imagenet',
    input_shape = (224,224,3)
)
from keras.models import Model,Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPool2D,Flatten
another_model = Sequential()
another_model.add(Flatten())
another_model.add(Dense(1024,activation='relu'))
another_model.add(Dense(512,activation='relu'))
another_model.add(Dense(20,activation='softmax'))
model = Sequential([base_model, another_model])
model.summary()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_2 (Sequential)            │ (None, 20)                  │      52,964,884 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 74,767,668 (285.22 MB)

 Trainable params: 74,733,236 (285.08 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_generator,validation_data = test_generator,epochs=50)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 191s 4s/step - accuracy: 0.0547 - loss: 7.3184 - val_accuracy: 0.0517 - val_loss: 2961846.2500
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.0556 - loss: 3.0868 - val_accuracy: 0.0500 - val_loss: 13368.9023
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.0523 - loss: 3.0044 - val_accuracy: 0.0467 - val_loss: 8611.3330
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0487 - loss: 3.0029 - val_accuracy: 0.0617 - val_loss: 557.3295
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.0370 - loss: 2.9981 - val_accuracy: 0.0467 - val_loss: 130.9072
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.0560 - loss: 3.0000 - val_accuracy: 0.0400 - val_loss: 187.6594
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.0407 - loss: 2.9986 - val_accuracy: 0.0400 - val_loss: 13.1691
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.0466 - loss: 2.9930 - val_accuracy: 0.0617 -

In [ ]:
model.evaluate(test_generator)

10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 864ms/step - accuracy: 0.2198 - loss: 4.7596


[4.682612419128418, 0.22499999403953552]

In [ ]:
model.save('food_model.h5')